In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F

import torchvision.datasets as dset
import torchvision.transforms as T
from torchvision import models
from torch.optim import lr_scheduler

import glob
from skimage import io, transform
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import time
import copy

from torchvision.models.resnet import BasicBlock
from torch.autograd import Variable

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [ ]:
USE_GPU = True

dtype = torch.float32

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 3

print('using device:', device)

In [ ]:
class GripForceDataset(Dataset):
    """Finger Grip Force dataset."""
    def __init__(self, root_dir, n_subj, n_exp_per_subj, transform=None):
        """
        Args:
            root_dir (string): Directory with all the images.
            n_subj (int): Number of test subjects.
            n_exp_per_subj (int): Number of experiments per test subject.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.N = 0
        self.images = np.zeros((0, 3), dtype=np.int)
        self.force = np.zeros((0, 2), dtype=np.float)
        for subject_number in n_subj:
            for experiment_number in n_exp_per_subj:
                img_path = '{}/{:02d}/{:02d}/frames_crop/'.format(root_dir, subject_number, experiment_number)
                force_path = '{}/{:02d}/{:02d}/newtons.csv'.format(root_dir, subject_number, experiment_number)
        
                # load image paths from current experiment
                n = len(glob.glob('{}*.png'.format(img_path)))
                experiment = np.zeros((n, 3), dtype=np.int)
                experiment[:,0] = subject_number
                experiment[:,1] = experiment_number
                experiment[:,2] = np.arange(n)
                
                # load force data from current experiment
                force = np.loadtxt(force_path, delimiter=',')
                
                # store image paths and force data (2 labels per image)
                self.images = np.concatenate((self.images, experiment), axis=0)
                self.force = np.concatenate((self.force, force), axis=0)
                self.N += n
                    
    def __len__(self):
        return self.N

    def __getitem__(self, idx):
        img_path = '{}/{:02d}/{:02d}/frames_crop/{:04d}.png'.format(self.root_dir, 
                                                                    self.images[idx, 0], 
                                                                    self.images[idx, 1], 
                                                                    self.images[idx, 2])
        frames = io.imread(img_path)
        frames = np.array(frames, dtype=np.float) / 255.0
        
        force = self.force[idx] / 30.0 - 0.5
        
        sample = {'frames': frames, 'force': force}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [ ]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        frames, force = sample['frames'], sample['force']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        frames = frames.transpose((2, 0, 1))
        return {'frames': torch.from_numpy(frames).float(),
                'force': torch.from_numpy(force).float()}

class Normalize(object):
    mean=[0.0, 0.0, 0.0]
    std=[1.0, 1.0, 1.0]
    
    # subj 1
    mean=[0.5695, 0.4747, 0.4095]
    std=[0.1545, 0.1762, 0.1931]

    # subj 2
#     mean = [0.5308, 0.4349, 0.3549]
#     std = [0.1436, 0.1596, 0.1730]

    # subj 3
#     mean = [0.5212, 0.4471, 0.4404]
#     std = [0.1569, 0.1885, 0.2102]

    # subj 4
#     mean = [0.5207, 0.4191, 0.4206]
#     std = [0.1518, 0.1816, 0.1995]

    # subj 5
#     mean = [0.5207, 0.4191, 0.4206]
#     std = [0.1518, 0.1816, 0.1995]

#     # subj 11
#     mean = [0.4948, 0.3974, 0.3697]
#     std = [0.2854, 0.2439, 0.2392]

    # subj 11-crop
    mean = [0.5232, 0.3953, 0.3532]
    std = [0.2882, 0.2319, 0.2124]
    
    def __call__(self, sample):
        frames, force = sample['frames'], sample['force']

        tr = T.Normalize(self.mean, self.std)
        
        frames = tr(frames)
        return {'frames': frames, 'force': force}

In [ ]:
# test Dataset class
root_dir = '/media/viktor/Samsung_T5/Research/dataset'
n_subj = [11]
n_exp_per_subj = range(1, 7)
# n_subj = [1]
# n_exp_per_subj = [3,1,2,4,5,6,7,8,9,10]

ds = GripForceDataset(root_dir, n_subj, n_exp_per_subj)

print('len={}'.format(ds.__len__()))

fig = plt.figure(figsize=(20, 20))
for i in range(5):
    sample = ds[0 + i]
    ax = plt.subplot(1, 6, i + 1)
    ax.set_title('frame#{}'.format(i))
    ax.axis('off')
    plt.imshow(sample['frames'])
    print(sample['frames'].shape)
plt.show()

In [ ]:
root_dir = '/media/viktor/Samsung_T5/Research/dataset'
# n_subj = [1]
# n_exp_per_subj = [10] #[3,1,2,4,5,6,7,8,9,10]

n_subj = [11]
n_exp_per_subj = [1] #range(1, 7)

normalize = True

# test Dataset with transform
if normalize == True:
    transform = T.Compose([ToTensor(), Normalize()])
else:
    transform = ToTensor()
    
transformed_dataset = GripForceDataset(root_dir, n_subj, n_exp_per_subj, transform=transform)

for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]

    print(i, sample['frames'].size(), sample['force'].size())

    if i == 3:
        break

In [ ]:
class ListSampler(sampler.Sampler):
    def __init__(self, indices, random=False):
        self.indices = indices
        self.random = random

    def __iter__(self):
        if self.random == True:
            np.random.shuffle(self.indices)
        return iter(self.indices)

    def __len__(self):
        return len(self.indices)

In [ ]:
### Split data into training, validation, and testing sets

batch_size = 64
num_workers = 64
val_split = .067
test_split = .1
shuffle_dataset = False # Only for train/val - test dataset is not shuffled
random_seed = 42

# Data indices
dataset_size = len(transformed_dataset)

test_size = int(np.floor(test_split * dataset_size))
val_size = int(np.floor(val_split * dataset_size))
train_size = dataset_size - val_size - test_size

# indices = list(range(test_size, dataset_size))

# if shuffle_dataset:
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)
# train_indices = indices[:train_size]
# val_indices = indices[train_size:]
# if not shuffle_dataset:
#     np.random.seed(random_seed)
#     np.random.shuffle(train_indices)
# test_indices = range(test_size)
    
# TODO: clean this mess
train_indices = list(range(train_size))
val_indices = list(range(train_size,train_size+val_size))
test_indices = list(range(train_size+val_size,dataset_size))
main_indices = list(range(dataset_size))

# # JUST FOR DEBUG PURPOSE
# batch_size = 3
# train_indices = train_indices[:6]
# val_indices = val_indices[:6]
# test_indices = test_indices[:6]

# Data Samplers
train_sampler = ListSampler(train_indices, random=True)
val_sampler = ListSampler(val_indices)
test_sampler = ListSampler(test_indices)
main_sampler = ListSampler(main_indices)

# Data Loaders
train_loader = torch.utils.data.DataLoader(transformed_dataset,
                                           batch_size=batch_size,
                                           num_workers=num_workers,
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(transformed_dataset,
                                         batch_size=batch_size,
                                         num_workers=num_workers,
                                         sampler=val_sampler)
test_loader = torch.utils.data.DataLoader(transformed_dataset,
                                          batch_size=batch_size,
                                          num_workers=num_workers,
                                          sampler=test_sampler)
main_loader = torch.utils.data.DataLoader(transformed_dataset,
                                          batch_size=batch_size,
                                          num_workers=num_workers,
                                          sampler=main_sampler)

dataloaders = {
    'train': train_loader,
    'val': val_loader,
    'test': test_loader,
    'main': main_loader
}

### Collecting the feature vectors

In [ ]:
# load pre-trained model
path = "/media/viktor/Samsung_T5/Research/models/11/transfer_28feb_resnet18_last.pth.tar"
model_loaded = models.resnet18()
model_loaded = torch.load(path)
model_loaded.eval()

In [ ]:
x = Variable(torch.randn(1, 3, 224, 224))
x = x.to("cuda")
model_loaded = model_loaded.to("cuda")
model_loaded(x)

In [ ]:
# make a model without the last layer(s)
model_cut = nn.Sequential(*list(model_loaded.children())[:-1])
model_cut(x).view(x.size()[0], -1)

In [ ]:
def features(loader, model):
    # storage for features
    storage = np.zeros((len(loader.dataset), 512))
                       
    model = model.to(device=device)
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        k = 0
        it = iter(loader)
        sample = next(it, None)
        while sample != None:
            x = sample['frames']
            sample = next(it, None)
            
            # move to device, e.g. GPU
            x = x.to(device=device, dtype=dtype)  
            
            # run the network on x
            features = model(x).view(x.size()[0], -1).cpu().numpy()
            
            # store features
            storage[k:k+batch_size,:] = features
            k += batch_size
            
    return storage

In [ ]:
storage = features(dataloaders['main'], model_cut)
storage.shape

In [ ]:
# save features
path = "/media/viktor/Samsung_T5/Research/dataset/11/01/"
np.savetxt(path + 'features.txt', storage)

### Dataset for LSTM

In [ ]:
class GripForceDatasetLSTM(Dataset):
    """Finger Grip Force dataset."""
    def __init__(self, root_dir, n_subj, n_exp_per_subj, k=20, transform=None):
        """
        Args:
            root_dir (string): Directory with all the images.
            n_subj (int): Number of test subjects.
            n_exp_per_subj (int): Number of experiments per test subject.
        """
        self.root_dir = root_dir
        self.N = 0
        self.force = np.zeros((0, 2), dtype=np.float)
        self.transform = transform
        self.features = np.zeros((0, 512), dtype=np.float)
        self.k = k
        self.samples = np.zeros(0, dtype=np.int)
        for subject_number in n_subj:
            for experiment_number in n_exp_per_subj:
                features_path = '{}/{:02d}/{:02d}/features.txt'.format(root_dir, subject_number, experiment_number)
                force_path = '{}/{:02d}/{:02d}/newtons.csv'.format(root_dir, subject_number, experiment_number)
            
                # load features and force data from current experiment
                features = np.loadtxt(features_path)
                force = np.loadtxt(force_path, delimiter=',')
                
                n = force.shape[0]
                
                # store image paths and force data (2 labels per image)
                self.features = np.concatenate((self.features, features), axis=0)
                self.force = np.concatenate((self.force, force), axis=0)
                
                # store number of the first frame for each sample of length k
                last_idx = self.force.shape[0] - k + 1
                first_idx = last_idx - (n - k + 1)
                self.samples = np.concatenate((self.samples, np.arange(first_idx, last_idx)), axis=0)
                self.N += n - k
    
    def __len__(self):
        return self.N

    def __getitem__(self, idx):
        i = self.samples[idx]
        
        features = self.features[i:i+self.k]
        
        force = self.force[i+self.k-1] / 30.0 - 0.5
        
        sample = {'features': features, 'force': force}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [ ]:
class ToTensorLSTM(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        frames, force = sample['features'], sample['force']

        return {'features': torch.from_numpy(frames).float(),
                'force': torch.from_numpy(force).float()}

In [ ]:
root_dir = '/media/viktor/Samsung_T5/Research/dataset'
n_subj = [11]
n_exp_per_subj = range(1, 7)
    
dataset = GripForceDatasetLSTM(root_dir, n_subj, n_exp_per_subj, k=25, transform=ToTensorLSTM())

for i in range(len(dataset)):
    sample = dataset[i]

    print(i, sample['features'].size(), sample['force'].size())
    
    if i == 3:
        break

### Dataloaders

In [ ]:
### Split data into training, validation, and testing sets

batch_size = 64
num_workers = 64
val_split = .067
test_split = .1
shuffle_dataset = False # Only for train/val - test dataset is not shuffled
random_seed = 42

# Data indices
dataset_size = len(dataset)

test_size = int(np.floor(test_split * dataset_size))
val_size = int(np.floor(val_split * dataset_size))
train_size = dataset_size - val_size - test_size

# indices = list(range(test_size, dataset_size))

# if shuffle_dataset:
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)
# train_indices = indices[:train_size]
# val_indices = indices[train_size:]
# if not shuffle_dataset:
#     np.random.seed(random_seed)
#     np.random.shuffle(train_indices)
# test_indices = range(test_size)
    
# TODO: clean this mess
train_indices = list(range(train_size))
val_indices = list(range(train_size,train_size+val_size))
test_indices = list(range(train_size+val_size,dataset_size))
main_indices = list(range(dataset_size))

# # JUST FOR DEBUG PURPOSE
# batch_size = 3
# train_indices = train_indices[:6]
# val_indices = val_indices[:6]
# test_indices = test_indices[:6]

# Data Samplers
train_sampler = ListSampler(train_indices, random=True)
val_sampler = ListSampler(val_indices)
test_sampler = ListSampler(test_indices)
main_sampler = ListSampler(main_indices)

# Data Loaders
train_loader = torch.utils.data.DataLoader(dataset,
                                           batch_size=batch_size,
                                           num_workers=num_workers,
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(dataset,
                                         batch_size=batch_size,
                                         num_workers=num_workers,
                                         sampler=val_sampler)
test_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=batch_size,
                                          num_workers=num_workers,
                                          sampler=test_sampler)
main_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=batch_size,
                                          num_workers=num_workers,
                                          sampler=main_sampler)

dataloaders = {
    'train': train_loader,
    'val': val_loader,
    'test': test_loader,
    'main': main_loader
}

### LSTM (feature vector input, force output)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    global loss_stat

    best_model_wts = copy.deepcopy(model.state_dict())
    best_mse = 1e10

    for epoch in range(num_epochs):
        
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            phase_since = time.time()
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for sample in dataloaders[phase]:
                inputs, labels = sample['features'], sample['force']
                
                current_batch_size = labels.size()[0]
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    model.hidden = model.init_hidden(current_batch_size)
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        loss_stat['train_'].append([epoch, loss.item()])

                # statistics
                running_loss += loss.item() * inputs.size(0)

#             epoch_loss = running_loss / dataset_sizes[phase]
            epoch_loss = running_loss / len(dataloaders[phase].sampler.indices)
            epoch_mse = epoch_loss * 30.0

            print('{} Loss: {:.4f} MSE: {:.4f}'.format(phase, epoch_loss, epoch_mse))
            now = time.time()
            time_elapsed = time.time() - since
            time_elapsed_phase = time.time() - phase_since
            print("Time: {:.0f}m {:.0f}s; Epoch {} in {:.0f}m {:.0f}s".format(
                time_elapsed // 60, time_elapsed % 60, phase,
                time_elapsed_phase // 60, time_elapsed_phase % 60
            ))

            # deep copy the model
            if phase == 'val' and epoch_mse < best_mse:
                best_mse = epoch_mse
                best_model_wts = copy.deepcopy(model.state_dict())

            loss_stat[phase].append([epoch, epoch_loss])
            
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val MSE: {:4f}'.format(best_mse))

    # load best model weights
    model_best = copy.deepcopy(model)
    model_best.load_state_dict(best_model_wts)
    return model_best, model

In [ ]:
class ForceEstimationLSTMT(nn.Module):

    def __init__(self, input_dim=512, hidden_dim=2, num_layers=1):
        super(ForceEstimationLSTMT, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # The linear layer that maps from hidden state space to tag space
#         self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

        self.hidden = self.init_hidden(64)

    def init_hidden(self, batch_size):
        # Before we've done anything, we dont have any hidden state.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        global device
        return (torch.randn(self.num_layers, batch_size, self.hidden_dim).to(device),
                torch.randn(self.num_layers, batch_size, self.hidden_dim).to(device))

    def forward(self, inputs):
        lstm_out, self.hidden = self.lstm(inputs, self.hidden)
        outputs = lstm_out[:,-1,:]
        return outputs

In [ ]:
lstm = ForceEstimationLSTMT(input_dim=512, hidden_dim=2, num_layers=1)

lstm = lstm.to(device)

criterion = nn.MSELoss()

# Observe that all parameters are being optimized
learning_rate = 5e-4
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
# optimizer = optim.SGD(lstm.parameters(), lr=learning_rate)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# set the bias for all forget gates to b
b = 1.
for names in lstm.lstm._all_weights:
    for name in filter(lambda n: "bias" in n,  names):
        bias = getattr(lstm.lstm, name)
        n = bias.size(0)
        start, end = n//4, n//2
        bias.data[start:end].fill_(b)

In [ ]:
loss_stat = {
    'train': [],
    'train_': [],
    'val': []
}

model_best, model_last = train_model(lstm, criterion, optimizer, exp_lr_scheduler, num_epochs=25)

for phase in ['train', 'val', 'train_']:
    loss_stat[phase] = np.array(loss_stat[phase])

In [ ]:
plt.figure(figsize=(20,15))
for phase in ['train', 'val']:
    plt.plot(loss_stat[phase][:,0], loss_stat[phase][:,1], '-', label=phase, linewidth=3)
# plt.plot(loss_stat['train_'][::10,1], '-', label=phase, linewidth=1)
plt.xlabel('Epoch', fontsize=40)
plt.ylabel('Loss', fontsize=40)
plt.title('Loss during training', fontsize=50)
plt.tick_params(labelsize=30)
plt.ylim(0,0.001)
plt.legend()
plt.grid()
plt.show()

In [ ]:
def data4visualization(loader, model, fps=30.0):
    y_true = []
    y_pred = []
    
    model = model.to(device=device)
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        k = 0
        it = iter(loader)
        sample = next(it, None)
        while sample != None:
            x, y = sample['features'], sample['force']
            sample = next(it, None)
            
            current_batch_size = y.size()[0]
            model.hidden = model.init_hidden(current_batch_size)
            
            # move to device, e.g. GPU
            x = x.to(device=device, dtype=dtype)  
            
            # predict
            у_pred_gpu = model(x)
            
            # store delta
            l = y.shape[0]
            y_pred_cpu = у_pred_gpu.data.cpu()
            y_pred.append(y_pred_cpu.numpy())
            y_true.append(y.numpy())
            k += l

    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
            
    y_pred = (y_pred + 0.5) * 30.0
    y_true = (y_true + 0.5) * 30.0
    
    t = np.arange(y_true.shape[0], dtype=np.float) / fps
    
    return t, y_true, y_pred

In [ ]:
linewidth = 5
def plot(t, y_true, y_pred, thumb=True, both=False, linewidth=2):
    plt.figure(figsize=(20,15))
    if thumb or both:
        plt.plot(t, y_true[:,0], 'r.-', linewidth=linewidth, label='Thumb, ground truth')
        plt.plot(t, y_pred[:,0], 'b.-', linewidth=linewidth, label='Thumb, prediction')
    if not thumb or both:
        plt.plot(t, y_true[:,1], 'g.-', linewidth=linewidth, label='Index, ground truth')
        plt.plot(t, y_pred[:,1], 'k.-', linewidth=linewidth, label='Index, prediction')
    plt.ylabel('Force [N]', fontsize=32)
    plt.xlabel('Time [sec]', fontsize=32)
    plt.grid()
    plt.legend(fontsize=30)
    plt.tick_params(labelsize=24)
    plt.show()

In [ ]:
t, y_true, y_pred = data4visualization(dataloaders['val'] , model_best)
plot(t, y_true, y_pred, thumb=True, both=False)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

finger = 0

print('%.2f' % r2_score(y_true[:,finger], y_pred[:,finger]))
print('%.2f' % mean_squared_error(y_true[:,finger], y_pred[:,finger]) ** .5)
print()

finger = 1

print('%.2f' % r2_score(y_true[:,finger], y_pred[:,finger]))
print('%.2f' % mean_squared_error(y_true[:,finger], y_pred[:,finger]) ** .5)

In [ ]:
t, y_true, y_pred = data4visualization(dataloaders['val'] , model_last)
plot(t, y_true, y_pred, thumb=True, both=False)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

finger = 0

print('%.2f' % r2_score(y_true[:,finger], y_pred[:,finger]))
print('%.2f' % mean_squared_error(y_true[:,finger], y_pred[:,finger]) ** .5)
print()

finger = 1

print('%.2f' % r2_score(y_true[:,finger], y_pred[:,finger]))
print('%.2f' % mean_squared_error(y_true[:,finger], y_pred[:,finger]) ** .5)

In [ ]:
plot(t, y_true, y_pred, thumb=True, both=False)

In [ ]:
# TODO: write an LSTM class with correct output (only the last prediction in time sequence)

In [ ]:
stop

In [ ]:


inputs = torch.randn(batch_size, 10, 512)
hidden = (torch.randn(1, batch_size, 2),
          torch.randn(1, batch_size, 2))

# out, hidden = lstm(inputs, hidden)
# print(out)
# print(hidden)

In [ ]:
for sample in train_loader:
    out, hidden = lstm(sample['features'], hidden)
    print(out)
    print(hidden)
    stop

### Incerting a LSTM layer in the model

In [ ]:
class ResNetLSTM(models.ResNet):
    def __init__(self, block, layers=[2, 2, 2, 2], num_classes=1000, params=None):
        super(ResNetLSTM, self).__init__(block, layers, num_classes)
        self.lstm = nn.LSTM(512, 2)
        self.h = (torch.randn(1, 1, 2), torch.randn(1, 1, 2))
                
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), 1, -1)
#         x = self.fc(x)
        x, h = self.lstm(x, self.h)
        self.h = h
        return x

lstm = ResNetLSTM(BasicBlock)
x = Variable(torch.randn(1, 3, 224, 224))
output = lstm(x)

In [ ]:
# load pre-trained model
path = "/media/viktor/Samsung_T5/Research/models/01/transfer_25dec.pth.tar"
model_loaded = models.resnet18()
model_loaded = torch.load(path)
model_loaded.eval()

In [ ]:
# load pretrained network weights
for name, param in model_loaded.state_dict().items():
    if name in lstm.state_dict():
        param = lstm.state_dict()[name]

In [ ]:
# non-matching params
for name, param in lstm.state_dict().items():
    if not name in model_loaded.state_dict():
        print(name, param.shape)